In [ ]:
import numpy as np
import sys
import os
import torch
from torch.autograd import Variable

n_train = 20000
# we're predicting J, H, and K magnitures from Teff, logg, and FeH

tmp = np.load('/Users/kareem/Proposals/binaries/apogee/JHK_photometry_isochrones.npz')
x_all = tmp['input_labels'].T
y_all = tmp['output_labels'].T

# scale labels
x_max = np.max(x_all, axis = 0)
x_min = np.min(x_all, axis = 0)
x_all = (x_all - x_min)/(x_max - x_min) - 0.5

x = x_all[:n_train]
y = y_all[:n_train]

dim_in = x.shape[1]

x = Variable(torch.from_numpy(x)).type(torch.FloatTensor)
y = Variable(torch.from_numpy(y), requires_grad=False).type(torch.FloatTensor)

# define neural network: One layer with 20 neurons. 
model = torch.nn.Sequential(
        torch.nn.Linear(dim_in, 20),
        torch.nn.Sigmoid(),
        torch.nn.Linear(20, 3),
)
loss_fn = torch.nn.L1Loss(size_average=True)

# define optimizer (play with the learning rate to see what works. )
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#-----------------------------------------------------------------------------
# convergence counter
current_loss = 1000.
count = 0
t = 0

# train the neural network
for i in range(int(1e5)):
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # print cost to monitor
    if i % 1000 == 0:
        print (i)
        print (loss.data[0])
    
model_numpy = []
for param in model.parameters():
    model_numpy.append(param.data.numpy())
    
w_array_0 = model_numpy[0]
b_array_0 = model_numpy[1]
w_array_1 = model_numpy[2]
b_array_1 = model_numpy[3]

# save parameters and remember how we scale the labels
np.savez("/Users/kareem/Proposals/binaries/NN_coeffs/NN_results_JHK.npz",
         w_array_0 = w_array_0,
         w_array_1 = w_array_1,
         b_array_0 = b_array_0,
         b_array_1 = b_array_1,
         x_max = x_max,
         x_min = x_min)
         
def get_JHK_photometry(input_labels, NN_coeffs_JHK):
    '''
    this is the function you'd use to predict the output (photometry)
    for a given set of input, *after* training the neural network. 
    NN_coeffs_JHK is the array of weights that parameterize the neural network
    input_labels: [Teff, logg, feh]
    '''
    w_array_0, w_array_1, b_array_0, b_array_1, x_min, x_max = NN_coeffs_JHK
    scaled_labels =  (input_labels - x_min)/(x_max - x_min) - 0.5
    first = np.dot(w_array_0, scaled_labels) + b_array_0
    second = np.dot(w_array_1, sigmoid(first)) + b_array_1
    return second
    
def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))
    

In [3]:
from __future__ import print_function
import torch

In [6]:
x = torch.Tensor(5,3)
print(x)


-1.2465e+05  4.5783e-41 -1.2465e+05
 4.5783e-41  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  4.6243e-44  0.0000e+00
 1.4908e-37  0.0000e+00 -1.2465e+05
[torch.FloatTensor of size 5x3]



In [14]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib
